In [1]:
from sqlite3 import connect
from pandas import read_sql_query

In [2]:
sql_create_product_table = """ CREATE TABLE IF NOT EXISTS product (
                                    product_id number primary key,
                                    name varchar not null,
                                    rrp number not null,
                                    available_from date not null
                                );
                                """
 
sql_create_orders_table = """ CREATE TABLE IF NOT EXISTS orders (
                                  order_id number primary key,
                                  product_id number not null,
                                  quantity number not null,
                                  order_price number not null,
                                  dispatch_date date not null,
                                  foreign key (product_id) references product(product_id)
                              );
                              """

insert_sql_products = """INSERT INTO product (product_id, name, rrp, available_from) VALUES
                        (101, 'Bayesian Methods for Nonlinear Classification and Regression', 94.95, date('now', 'weekday 3', '-7 day')),
                        (102, '2018 in Review preorder', 21.95, date('now','+1 year')),
                        (103, 'Learn Python in Ten Minutes', 2.15, date('now','-3 month')),
                        (104, 'sports almanac 1999-2049', 3.38, date('now','-2 year')),
                        (105, 'finance for dummies', 84.99, date('now','-1 year'));
                        """

insert_sql_orders = """INSERT INTO orders (order_id, product_id, quantity, order_price, dispatch_date) VALUES
                        (1000, 101, 1, 90.00, date('now','-2 month')),
                        (1001, 103, 1, 1.15, date('now','-40 day')),
                        (1002, 101, 10, 90.00, date('now','-11 month')),
                        (1003, 104, 11, 3.38, date('now','-6 month')),
                        (1004, 105, 11, 501.33, date('now','-2 year'));
                        """

In [3]:
con = connect(":memory:")
cur = con.cursor()
cur.execute(sql_create_product_table)
cur.execute(sql_create_orders_table)
cur.execute(insert_sql_orders)
cur.execute(insert_sql_products)

In [4]:
df_product = read_sql_query("select * from product", con)
df_orders = read_sql_query("select * from orders", con)

In [5]:
df_product

,product_id,name,rrp,available_from
0,101,Bayesian Methods for Nonlinear Classification ...,94.95,2018-04-25
1,102,2018 in Review preorder,21.95,2019-04-26
2,103,Learn Python in Ten Minutes,2.15,2018-01-26
3,104,sports almanac 1999-2049,3.38,2016-04-26
4,105,finance for dummies,84.99,2017-04-26


In [6]:
df_orders

,order_id,product_id,quantity,order_price,dispatch_date
0,1000,101,1,90.00,2018-02-26
1,1001,103,1,1.15,2018-03-17
2,1002,101,10,90.00,2017-05-26
3,1003,104,11,3.38,2017-10-26
4,1004,105,11,501.33,2016-04-26


##### Write an sql query to find books that have
- sold fewer than 10 copies 
- sold in the last year, 
- excluding books that have been available for less than 1 month.

In [8]:
sql_query = """
                     SELECT p.*, sum(quantity) as copies_sold 
                     FROM product p 
                     JOIN orders o on o.product_id == p.product_id
                     WHERE o.dispatch_date > date('now', '-1 year')
                     AND p.available_from < date('now', '-1 month') 
                     GROUP BY o.product_id 
                     HAVING copies_sold < 10
                     """
read_sql_query(sql_query, con)

,product_id,name,rrp,available_from,copies_sold
0,103,Learn Python in Ten Minutes,2.15,2018-01-26,1


We assume `dispatch_date` to be the sold date but also consider the possibility that items that can dispatch in the future and therefore count them as sold.